In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# 사용예시
with open('credentials.txt', 'r') as file:
    sender_email = file.readline().strip()  # 발신자 이메일 주소
    password = file.readline().strip()  # 발신자 이메일 비밀번호

with open('receivers.txt', 'r') as file:
    receivers = file.read().splitlines()  # 수신자 이메일 주소 목록

def load_template(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:  # UTF-8 인코딩으로 파일을 엽니다
            return file.read()
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return None
    except UnicodeDecodeError:
        print(f"파일 인코딩 문제: {file_path}")
        return None

def create_message(sender_email, receiver_email, subject, text, html):
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    message.attach(MIMEText(text, "plain"))
    message.attach(MIMEText(html, "html"))
    return message

# 기존 send_weather_email 함수 내에서 변경된 부분
text_template = load_template('text.txt')
html_template = load_template('html.html')
subject = "서울 지역 폭우 예보 알림"
location = "서울"
weather_condition = "폭우"

# 문자열 포매팅을 사용하여 실제 값으로 대체

if text_template is not None and html_template is not None:
    for receiver_email in receivers:
        # 문자열 포매팅을 적용하여 실제 값으로 대체
        text = text_template.format(location=location, weather_condition=weather_condition)
        html = html_template.format(location=location, weather_condition=weather_condition)
        
        message = create_message(sender_email, receiver_email, subject, text, html)
        try:
            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message.as_string())
                print(f"{receiver_email}로 이메일 전송 성공")
        except Exception as e:
            print(f"이메일 전송 실패: {e}")

